# Building and evaluating transformer models and their optimal hyperparameters
#### Evaluate and then pick the one I like to implement in my app

In [3]:
!git clone https://github.com/hannahishimwe/FakeProfileDetection.git

fatal: destination path 'FakeProfileDetection' already exists and is not an empty directory.


In [4]:
!pwd

/content


In [5]:
# %%capture
!pip install pandas transformers datasets torch transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer


In [7]:
PATH_TO_TRAINING_CSV = "./FakeProfileDetection/dev/csv/structured_train.csv"
PATH_TO_TEST_CSV = "./FakeProfileDetection/dev/csv/structured_test.csv"

training_df = pd.read_csv(PATH_TO_TRAINING_CSV)
testing_df = pd.read_csv(PATH_TO_TEST_CSV)
pd.set_option('display.max_colwidth', None)
training_df.head(50)

,screen_name,text,account.type,class_type
0,imranyebot,YEA now that note GOOD,bot,others
1,zawvrk,Listen to This Charming Man by The Smiths https://t.co/r12OIXkfUO,human,human
2,zawarbot,wish i can i would be seeing other hoes on the worst part,bot,others
3,ahadsheriffbot,The decade in the significantly easier schedule I Don't like h…,bot,others
4,kevinhookebot,"""Theim class=\""alignnone size-full wp-image-6017 \"" src=\""https://t.co/LiAsQsbs99\"" alt=\""\"" /&gt;\r\n&lt;p id=\""dyD\""&gt;&lt;img class=\""alignnone size-full wp-image-6047 \"" src=""",bot,rnn
5,dril,matrix 2: pedaphile killer,human,human
6,whalefakes,the world needs more whale stories. I would love to know what whalefacts are hiding in them.,bot,gpt2
7,imranyebot,the tweet of time I don’t mean It’s got us free lacroix,bot,others
8,ninjasexparty,nobody:\r\n\r\nNSP: penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis penis pe,human,human
9,narendramodi,"Had a wonderful interaction with leaders and MPs across party lines this evening. \r\n\r\nWe look forward to a productive Parliament session, where people-centric and development oriented issues would be discussed. https://t.co/EsNze39bSA",human,human


In [8]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 13.1 MB/s eta 0:00:00


In [9]:
from FakeProfileDetection.dev.CleanDatasets import CleanDatasets
from datasets import Dataset

columns_to_rename_dict = {"screen_name": "username", "account.type": "is_human"}
columns_to_drop_list = ["class_type"]
column_to_binary = "account.type"


training_cleaner = CleanDatasets(training_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_training_df = training_cleaner.clean_df()
testing_cleaner = CleanDatasets(testing_df, columns_to_rename_dict, columns_to_drop_list, column_to_binary)
clean_testing_df = testing_cleaner.clean_df()
train_dataset = Dataset.from_pandas(clean_training_df)
test_dataset = Dataset.from_pandas(clean_testing_df)
train_dataset


Dataset({
    features: ['username', 'text', 'is_human'],
    num_rows: 20712
})

In [10]:
#initialise variables - build a config for different models i can use

MODEL_NAME = "FacebookAI/roberta-base"
TOKENIZER_MODEL_NAME = "roberta-base"
MODEL = RobertaForSequenceClassification
TOKENIZER = RobertaTokenizer



In [11]:
# Tokenize the texts - replace 'text' in each column with vectors, attention mask and label so machines can work with it
tokenizer = RobertaTokenizer.from_pretrained(TOKENIZER_MODEL_NAME)

# Tokenize the texts
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/20712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

In [12]:
train_tokenized = train_tokenized.rename_column("is_human", "labels")
test_tokenized = test_tokenized.rename_column("is_human", "labels")

train_tokenized = train_tokenized.remove_columns(["username", "text"])
test_tokenized = test_tokenized.remove_columns(["username", "text"])


In [13]:

train_tokenized[0].keys()

dict_keys(['labels', 'input_ids', 'attention_mask'])

In [14]:
pip show torch

Name: torch
Version: 2.5.1+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [15]:
#configurate the trainer
"""got this warning, just means train: Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference."""
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=MODEL.from_pretrained(MODEL_NAME),
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized
)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hannah-ishimwe (hannah-ishimwe-king-s-college-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.459000,0.482826
2,0.343800,0.329222
3,0.285000,0.421888


TrainOutput(global_step=7767, training_loss=0.3825558991111106, metrics={'train_runtime': 6579.2437, 'train_samples_per_second': 9.444, 'train_steps_per_second': 1.181, 'total_flos': 1.634866853584896e+16, 'train_loss': 0.3825558991111106, 'epoch': 3.0})